In [133]:
from datetime import datetime, timedelta
import urllib.request
import requests
import os

In [179]:
def getDataFromDate(start_date):
    day_to_be_scraped = start_date
    today = datetime.today()
    days_without_data = []
    
    bx_url = 'ftp://ftp.nasdaqtrader.com/files/shortsaledata/daily/bx/NQBXshvol'
    psx_url = 'ftp://ftp.nasdaqtrader.com/files/shortsaledata/daily/psx/NPSXshvol'
    finra_url = 'http://regsho.finra.org/CNMSshvol'
    
    if (not os.path.isdir('B')):
        os.mkdir('B')
    if (not os.path.isdir('P')):
        os.mkdir('P')
    if (not os.path.isdir('F')):
        os.mkdir('F')

    while (day_to_be_scraped <= today):
        filename = (day_to_be_scraped).strftime('%Y%m%d') + '.txt'
        try:

            # must be done with requests. 403 Forbidden otherwise
            # TODO: clean the files that are failed http requests
            finra_query = requests.get(finra_url+filename)
            with open('F/'+filename, 'wb') as outfile:
                outfile.write(finra_query.content)


            with closing(request.urlopen(bx_url+filename)) as bx_query:
                with open('B/'+filename, 'wb') as f:
                    shutil.copyfileobj(bx_query, f)

            with closing(request.urlopen(psx_url+filename)) as psx_query:
                with open('P/'+filename, 'wb') as f:
                    shutil.copyfileobj(psx_query, f)
                

        except:
            print('no data for '+ filename)
            
            #kinda hacky
            days_without_data +=[filename]
            

        day_to_be_scraped += timedelta(days=1)
        
    return days_without_data


In [180]:
today = datetime.today()
day_to_be_scraped = today - timedelta(days=15)

files_to_be_removed = getDataFromDate(day_to_be_scraped)

no data for 20210130.txt
no data for 20210131.txt
no data for 20210206.txt
no data for 20210207.txt
no data for 20210213.txt


In [181]:
# clean finra files that have no data
for finra_file in files_to_be_removed:
    os.remove('F/'+finra_file)

In [ ]:
!pip3 install python-crontab